# Temporal Difference Control: Expected Sarsa

An implementation of "expected sarsa"  using a gridworld.


The gridworld has the shape(3,4) with a winning state "w"(0,3), and a lossing state "l"(1,3), a non valid state "x"(2,1) and a start state s(3,0)

|  |  |  |  |
|---|---|---|---|
|  |  |  | w |
|  |  |  | l |
|  | x |  |  |
| s |  |  |  |

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict

import grid_world

### Disccount factor and step size

In [2]:

GAMMA = 0.9
ALPHA =0.1

### Auxiliary function to display the values of a policy after finishing iterative policy evaluation

In [3]:
def print_values(V,grid):
    for i in range(grid.width):
        print("--------------------------")
        for j in range(grid.height):
            v = V.get((i,j),0)
            if v >= 0:
                print(" %.2f|" % v, end="")
            else:
                print("%.2f|" % v, end="")
        print("")

### Auxiliary function to display a stochastic policy

In [4]:
def print_policy(P,grid):
    for i in range(grid.width):
        print("---------------------------")
        for j in range(grid.height):
            a = P.get((i,j),' ')
            if isinstance(a,dict):
                a = list(a)[0]
            print("  %s  |" % a, end="")
        print("")

### From or defined grid world file, import a negative grid ,retrieve all actions and states and print grid rewards
Negative grid is used to encourage the agent to find a shortest path to the goal

In [5]:
grid = grid_world.Grid.standard_grid()
states = grid.all_states()
actions = list(set([action   for action_tup in grid.actions.values() for action in action_tup]))

In [6]:
def argmax_dict(dictionary):
    # returns the argmax key and the max value from a dictionary
    # will be used for policy improvement from Q
    max_key = None
    max_val = float("-inf")
    
    for k,v in dictionary.items():
        if v > max_val:
            max_val = v
            max_key = k
            
    return max_key,max_val
        
argmax_dict({"a":1,"b":2})

('b', 2)

In [7]:
actions

['R', 'L', 'U', 'D']

In [8]:
def epsilon_greedy_action(Q,state,epsilon=0.1):
    # choose an action using epsilon-greedy strategy
    probability = np.random.random()
    result = 0
    
    if probability < epsilon:
        #explore
        result = np.random.choice(actions)
    else: 
        #exploit
        result = argmax_dict(Q[state])[0]
        
    return result

In [9]:
print("Rewards of grid")
print_values(grid.rewards,grid)

Rewards of grid
--------------------------
 0.00| 0.00| 0.00| 1.00|
--------------------------
 0.00| 0.00| 0.00|-1.00|
--------------------------
 0.00| 0.00| 0.00| 0.00|


### Initialize  policy

In [10]:
policy = {(2,0):'U',
         (1,0):'U',
         (0,0):'R',
         (0,1):'R',
         (0,2):'R',
         (1,2):'R',
         (2,1):'R',
         (2,2):'R',
         (2,3):'U'}

print_policy(policy,grid)

---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  R  |     |
---------------------------
  U  |  R  |  R  |  U  |


In [16]:
def expected_sarsa(grid,policy,episodes,gamma =1,alpha=1):
    Q = dict()
    
    for state in grid.all_states():
        Q[state] = dict()
        for action in actions:
            Q[state][action] = 0
            
            

    for episode in range(1,episodes +1):
        epsilon  = 1/episode #0.5
        finished = False
        
        s = (2,0)
        grid.set_state(s)
        
        
        while not finished:
            a = epsilon_greedy_action(Q,s,epsilon)
            r = grid.move(a)    
            s1 = grid.current_state()
            
            # calcuate expected Q(s1,a) with respect to a
            ## calculate the probability of every action for current state, using epsilon-greedy
            policy_action_probs = defaultdict(lambda:(epsilon/len(actions)))
            policy_action_probs[argmax_dict(Q[s1])[0]] = (1-epsilon) +(epsilon/len(actions))
            expected_Qs1 = 0
            for action in actions:
                expected_Qs1 += (policy_action_probs[action]*Q[s1][action])
                
            # expected sarsa update
            Q[s][a] = Q[s][a] + alpha*(r + (gamma*expected_Qs1) - Q[s][a])
            
            # Q learning update left for comparison
            #new_state_argmax = argmax_dict(Q[s1]) # dictionary with (action,value) meaning returns the best action and its value
            #Q[s][a] = Q[s][a] + alpha*(r + (gamma*new_state_argmax[1]) - Q[s][a])
            
            #sarsa update left for comparison to q LEARNING
            #Q[s][a] = Q[s][a] + alpha*(r +(gamma*Q[s1][a1]) - Q[s][a])
            
            finished =grid.game_over()
                
            s = s1
            
    for s in policy.keys():
        state_greedy_action = argmax_dict(Q[s])
        
        if not state_greedy_action[0] is None:
            policy[s] = state_greedy_action[0]
            
    return policy,Q

policy,Q = expected_sarsa(grid,policy,100000,GAMMA,ALPHA)

In [17]:
policy

{(0, 0): 'R',
 (0, 1): 'R',
 (0, 2): 'R',
 (1, 0): 'U',
 (1, 2): 'U',
 (2, 0): 'U',
 (2, 1): 'L',
 (2, 2): 'U',
 (2, 3): 'D'}

In [18]:
print("Policy")
print_policy(policy,grid)

Policy
---------------------------
  R  |  R  |  R  |     |
---------------------------
  U  |     |  U  |     |
---------------------------
  U  |  L  |  U  |  D  |


In [19]:
V = defaultdict(lambda:0)
for state in policy.keys():
    V[state] = Q[state][policy[state]]
    print("state  |policy action| state value")
    print(state,"|      ",policy[state] , "    |", V[state] )
    

state  |policy action| state value
(2, 0) |       U     | 0.6560828553230844
state  |policy action| state value
(1, 0) |       U     | 0.7289858018075595
state  |policy action| state value
(0, 0) |       R     | 0.8099894711397438
state  |policy action| state value
(0, 1) |       R     | 0.899994092187002
state  |policy action| state value
(0, 2) |       R     | 0.9999999999999996
state  |policy action| state value
(1, 2) |       U     | 0.3505114912840074
state  |policy action| state value
(2, 1) |       L     | 0.10742530922018076
state  |policy action| state value
(2, 2) |       U     | 0.0
state  |policy action| state value
(2, 3) |       D     | 0


In [20]:
print_values(grid=grid,V=V)

--------------------------
 0.81| 0.90| 1.00| 0.00|
--------------------------
 0.73| 0.00| 0.35| 0.00|
--------------------------
 0.66| 0.11| 0.00| 0.00|


## Conclusions
* Expected sarsa converges faster than Q-learning and it's less sensible to hyperparameters
* Expected sarsa converges to a more relastic Q action value function